# Deploy the finetuned vicuna model on Amazon SageMaker with djl server batch

## Create a SageMaker Model for Deployment
As a first step, we'll import the relevant libraries and configure several global variables such as the hosting image that will be used nd the S3 location of our model artifacts

In [32]:
import sagemaker
from sagemaker.model import Model
from sagemaker import serializers, deserializers
from sagemaker import image_uris
import boto3
import os
import time
import json
import jinja2
from pathlib import Path

In [33]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3") # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime") # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment() # jinja environment to generate model configuration templates

In [34]:
# lookup the inference image uri based on our current region
djl_inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

In [35]:
pretrained_model_location = "s3://sagemaker-us-west-2-687912291502/llama/output/2023-05-10-12-43-02/llama_out/"# Change to the model artifact path in S3 which we get from the fine tune job
print(f"Pretrained model will be downloaded from ---- > {pretrained_model_location}")

Pretrained model will be downloaded from ---- > s3://sagemaker-us-west-2-687912291502/llama/output/2023-05-10-12-43-02/llama_out/


## Build the inference contianer image

In [52]:
%%writefile Dockerfile.inference
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

## Install transfomers version which support LLaMaTokenizer
#RUN python3 -m pip install git+https://github.com/huggingface/transformers.git@68d640f7c368bcaaaecfc678f11908ebbd3d6176
## Install transfomers version which support vicuna v1.1 LLaMaTokenizer
#RUN python3 -m pip install transformers==4.29.0
RUN python3 -m pip install transformers==4.28.0

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

Overwriting Dockerfile.inference


In [53]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [54]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-vicuna-inference-severbatch-demo"

In [55]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} -f Dockerfile.inference .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  66.92MB
Step 1/6 : From 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117
 ---> a63b0d190846
Step 2/6 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> 311767347229
Step 3/6 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 744d3881b2eb
Step 4/6 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 114266d93af2
Step 5/6 : RUN python3 -m pip install transformers==4.28.0
 ---> Running in edf2603013fa
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 78.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.0
    Uninstalling transformers-4.26.0:
      Successfully uninstalled transformers-4.26.0
Removing intermediate container edf2603013fa
 ---> 73084c6782b3
Step 6/6 : ENV NVIDIA_VISIBLE_DEVICES="all"
 ---> Running in bd3307037b55
Removing intermediate container bd3307037b55
 ---> 32bdede2252d
Successfully built 32bdede2252d
Successfully 

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [56]:
## The image uri which is build and pushed above
inference_image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account_id, region, repo_name)
inference_image_uri

'687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-vicuna-inference-severbatch-demo:latest'

## Deploying a Large Language Model using deepspeed engine
The DJL Inference Image which we will be utilizing ships with a number of built-in inference handlers for a wide variety of tasks including:
- `text-generation`
- `question-answering`
- `text-classification`
- `token-classification`

You can refer to this [GitRepo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python) for a list of additional handlers and available NLP Tasks. <br>
These handlers can be utilized as is without having to write any custom inference code. We simply need to create a `serving.properties` text file with our desired hosting options and package it up into a `tar.gz` artifact.

Lets take a look at the `serving.properties` file that we'll be using for our first example

In [57]:
!mkdir deepspeed_src

mkdir: cannot create directory ‘deepspeed_src’: File exists


In [58]:
%%writefile deepspeed_src/serving.template
engine=DeepSpeed
option.entryPoint=model.py
option.tensor_parallel_degree=1
option.model_id=eachadea/vicuna-7b-1.1
batch_size=2
max_batch_delay=5000

Overwriting deepspeed_src/serving.template


In [67]:
%%writefile deepspeed_src/model.py
from djl_python import Input, Output
import os
import logging
import math
import deepspeed
import torch
import torch.distributed as dist
import sys
import subprocess
import time
import transformers
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers.models.llama.tokenization_llama import LlamaTokenizer

print("transformers version=="+transformers.__version__)

predictor = None

#for deepspeed engine

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    #logging.info(f"Loading model in {model_location}")
    
    print("----------tensor parallel is {0}---------".format(tensor_parallel))
    #model_location="decapoda-research/llama-7b-hf"
    logging.info(f"Loading model in {model_location}")

    model = AutoModelForCausalLM.from_pretrained(model_location, torch_dtype=torch.float16)
    #tokenizer = LlamaTokenizer.from_pretrained(model_location)
    tokenizer = LlamaTokenizer.from_pretrained(model_location, torch_dtype=torch.float16)

    print("----------model dtype is {0}---------".format(model.dtype))
    model = deepspeed.init_inference(
        model,
        mp_size=tensor_parallel,
        #dtype=model.dtype,
        dtype=torch.half,
        #dtype=torch.int8,
        replace_method="auto",
        replace_with_kernel_inject=True,
        #replace_method="auto",
        #replace_with_kernel_inject=True,
    )
    
    local_rank = int(os.getenv("LOCAL_RANK", "0"))
    generator = pipeline(
        task="text-generation", model=model, tokenizer=tokenizer, device=local_rank
    )
    return generator, model, tokenizer

'''
def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    #logging.info(f"Loading model in {model_location}")
    
    print("----------tensor parallel is {0}---------".format(tensor_parallel))
    #model_location="decapoda-research/llama-7b-hf"
    logging.info(f"Loading model in {model_location}")

    model = AutoModelForCausalLM.from_pretrained(model_location, torch_dtype=torch.float16, device_map="balanced_low_0")
    #tokenizer = LlamaTokenizer.from_pretrained(model_location)
    tokenizer = LlamaTokenizer.from_pretrained(model_location, torch_dtype=torch.float16)

    print("----------model dtype is {0}---------".format(model.dtype))
    generator = pipeline(
        task="text-generation", model=model, tokenizer=tokenizer,
    )
    return generator, model, tokenizer
'''

def handle(inputs: Input) -> None:
    global predictor, model, tokenizer
    try:
        if not predictor:
            predictor,model,tokenizer = load_model(inputs.get_properties())

        print(inputs)
        if inputs.is_empty():
            # Model server makes an empty call to warmup the model on startup
            return None
        
        if inputs.is_batch():
            logging.info(f"Dynamic batching size: {inputs.get_batch_size()}.")
            batch = inputs.get_batches()
            print(batch)
            tmp_inputs = []
            for _, item in enumerate(batch):
                tmp_item = item.get_as_json()
                tmp_inputs.append(tmp_item.get("input"))
            
            result = predictor(tmp_inputs)
            outputs = Output()
            for i in range(len(result)):
                outputs.add(result[i], key="generate_text", batch_index=i)
            return outputs
        else:
            inputs = inputs.get_as_json()
            if not inputs.get("input"):
                return Output().add_as_json({"code":-1,"msg":"input field can't be null"})


            #input data
            data = inputs.get("input")
            params = inputs.get("params",{})

            #predictor
            result = predictor(data, **params)

            #return
            return Output().add({"code":0,"msg":"ok","data":result})
    except Exception as e:
        return Output().add_as_json({"code":-1,"msg":e})


Overwriting deepspeed_src/model.py


In [68]:
%%writefile deepspeed_src/requirements.txt
protobuf==3.20
accelerate=>0.17.0

Overwriting deepspeed_src/requirements.txt


In [69]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("deepspeed_src/serving.template").open().read())
Path("deepspeed_src/serving.properties").open("w").write(template.render(s3url=pretrained_model_location))
!pygmentize accelerate_src/serving.properties | cat -n

     1	engine=Python
     2	option.entryPoint=djl_python.huggingface
     3	#option.s3url=s3://sagemaker-us-west-2-687912291502/llama/output/2023-05-10-12-43-02/llama_out/
     4	#option.model_id=pinkmanlove/llama-7b-hf
     5	#option.model_id=eachadea/vicuna-7b-1.1
     6	option.model_id=TheBloke/vicuna-7B-1.1-HF
     7	option.task=text-generation
     8	option.device_map=auto
     9	option.load_in_8bit=TRUE


There are a few options specified here. Lets go through them in turn<br>
1. `engine` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the [DJL Python Engine](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python)
2. `option.entryPoint` - specifies the entrypoint code that will be used to host the model. djl_python.huggingface refers to the `huggingface.py` module from [djl_python repo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python).  
3. `option.s3url` - specifies the location of the model files. Alternativelly an `option.model_id` option can be used instead to specifiy a model from Hugging Face Hub (e.g. `EleutherAI/gpt-j-6B`) and the model will be automatically downloaded from the Hub. The s3url approach is recommended as it allows you to host the model artifact within your own environment and enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance 
4. `option.task` - This is specific to the `huggingface.py` inference handler and specifies for which task this model will be used
5. `option.device_map` - Enables layer-wise model partitioning through [Hugging Face Accelerate](https://huggingface.co/docs/accelerate/usage_guides/big_modeling#designing-a-device-map). With `option.device_map=auto`, Accelerate will determine where to put each **layer** to maximize the use of your fastest devices (GPUs) and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM). Even if the model is split across several devices, it will run as you would normally expect.
6. `option.load_in_8bit` - Quantizes the model weights to int8 thereby greatly reducing the memory footprint of the model from the initial FP32. See this [blog post](https://huggingface.co/blog/hf-bitsandbytes-integration) from Hugging Face for additional information 

For more information on the available options, please refer to the [SageMaker Large Model Inference Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html)

Our initial approach here is to utilize the built-in functionality within Hugging Face Transformers to enable Large Language Model hosting. 

We place the `serving.properties` file into a tarball and upload it to S3

In [70]:
!tar czvf acc_model.tar.gz deepspeed_src/ 

deepspeed_src/
deepspeed_src/.ipynb_checkpoints/
deepspeed_src/model.py
deepspeed_src/serving.template
deepspeed_src/serving.properties
deepspeed_src/requirements.txt


In [71]:
s3_code_prefix = "llama/deploy/code"

code_artifact = sess.upload_data("acc_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/llama/deploy/code/acc_model.tar.gz


## Deploy Model to a SageMaker Endpoint
With a helper function we can now deploy our endpoint and invoke it with some sample inputs

In [72]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"vicuna-7B")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": code_artifact
    }
)
model_arn = create_model_response["ModelArn"]
print(model_arn)

endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.4xlarge",
            "InitialInstanceCount": 1,
        }
    ]
)
print(endpoint_config_response)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response)

arn:aws:sagemaker:us-west-2:687912291502:model/vicuna-7b-2023-05-19-07-53-30-962
{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:687912291502:endpoint-config/vicuna-7b-2023-05-19-07-53-30-962-config', 'ResponseMetadata': {'RequestId': 'a9dde413-92a6-4f9e-83a7-0b7fe179c80c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a9dde413-92a6-4f9e-83a7-0b7fe179c80c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '121', 'date': 'Fri, 19 May 2023 07:53:31 GMT'}, 'RetryAttempts': 0}}
{'EndpointArn': 'arn:aws:sagemaker:us-west-2:687912291502:endpoint/vicuna-7b-2023-05-19-07-53-30-962-endpoint', 'ResponseMetadata': {'RequestId': '0125d8d0-9627-4aa9-b0a9-82bff741f5b8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0125d8d0-9627-4aa9-b0a9-82bff741f5b8', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'date': 'Fri, 19 May 2023 07:53:32 GMT'}, 'RetryAttempts': 0}}


Let's run an example with a basic text generation prompt Large model inference is

In [75]:
import boto3
import json
import uuid
client = boto3.client('sagemaker-runtime')
endpoint_name = 'vicuna-7B-2023-05-19-07-53-30-962-endpoint'
payload1 = '{"input" : "a happy weekend with my family", \
            "parameters": { "max_length": 200, "temperature": 0.6 }  \
           }'

payload2 = '{"input" : "I\'d likt to go to HongKong", \
            "parameters": { "max_length": 200, "temperature": 0.6 }  \
           }'
encoded_inp = (payload1).encode("utf-8")
response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Accept='application/json',
        Body=payload1
)
response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Accept='application/json',
        Body=payload2
)


#predictor.predict({ 
#                    "inputs" : "large model inference is", 
#                    "parameters": { "max_length": 100, "temperature": 0.8 }
#                })

In [76]:
result = response['Body'].read()
result = json.loads(result)
print(result)


{'code': 0, 'msg': 'ok', 'data': [{'generated_text': 'a happy weekend with my family and friends.\nI hope you all have a great weekend'}]}


In [34]:
print(predictor.predict({ "inputs":
                                """Message: Support has been terrible for 2 weeks...
                                Sentiment: Negative
                                ###
                                Message: I love your API, it is simple and so fast!
                                Sentiment: Positive
                                ###
                                Message: GPT-J has been released 12 months ago.
                                Sentiment: Neutral
                                ###
                                Message: The responsiveness of your team has been amazing, thank you so much!
                                Sentiment:""",
                      "parameters": { "max_length": 50, "temperature": 0.5 }
                     }
                    )[0]["generated_text"])

Message: Support has been terrible for 2 weeks...
                                Sentiment: Negative
                                ###
                                Message: I love your API, it is simple and so fast!
                                Sentiment: Positive
                                ###
                                Message: GPT-J has been released 12 months ago.
                                Sentiment: Neutral
                                ###
                                Message: The responsiveness of your team has been amazing, thank you so much!
                                Sentiment: Pos


Finally Let's do a quick benchmark to see what kind of latency we can expect from this model

In [29]:
predictor.predict({ 
                    "inputs" : "a happy weekend with my family", 
                    "parameters": { "max_length": 150, "temperature": 0.5 }
                })

[{'generated_text': "a happy weekend with my family.\nI'm so glad you had a great weekend! I hope you have a great week!\nI'm so glad you had a great weekend! I hope you have a great week! I'm so glad you had a great weekend! I hope you have a great week!\nI'm so glad you had a great weekend! I hope you have a great week!\nI'm so glad you had a great weekend! I hope you have a great week!\nI'm so glad you had a great weekend! I hope you have a great week! I'm so glad you had a great weekend! I hope you have a"}]

In [ ]:
# Clean up the endpoint before proceeding
predictor.delete_endpoint()

## Reference

[sagemaker-hosting/Large-Language-Model-Hosting/](https://github.com/aws-samples/sagemaker-hosting/tree/main/Large-Language-Model-Hosting)